In [3]:
from spacy.tokens import DocBin
import pandas as pd
import string
import spacy
import gensim
from gensim.utils import simple_preprocess

In [ ]:
train_data = pd.read_excel("severity_training.xlsx")
valid_data = pd.read_excel("severity_training.xlsx")

In [ ]:
train_data = list(train_data.itertuples(index = False, name = None))
valid_data = list(valid_data.itertuples(index = False, name = None))

In [ ]:
def remove_punctuation(input_string):
        translator = str.maketrans("","", string.punctuation)
        return input_string.translate(translator)

def note_replacer(note_value):
    clean_note = ""  
    note_value = remove_punctuation(note_value)    
    for char in note_value:
        if char.isalpha():
            clean_note += char.lower()
        else:
            clean_note += char
    return clean_note

In [ ]:
for value in train_data:
    index = train_data.index(value)
    value = (note_replacer(value[0]), value[1])
    train_data[index] = value

for value in valid_data:
    index = valid_data.index(value)
    value = (note_replacer(value[0]), value[1])
    valid_data[index] = value

In [ ]:
def make_docs(data):
    docs = []
    for doc, label in nlp.pipe(data, as_tuples = True):
        if label == "low":
            doc.cats["low"] = 1
            doc.cats["moderate"] = 0
            doc.cats["high"] = 0
        elif label == "moderate":
            doc.cats["low"] = 0
            doc.cats["moderate"] = 1
            doc.cats["high"] = 0
        elif label == "high":
            doc.cats["low"] = 0
            doc.cats["moderate"] = 0
            doc.cats["high"] = 1
        docs.append(doc)
    return(docs)

In [ ]:
nlp = spacy.load("en_core_web_lg")

In [ ]:
num_texts = 1000

train_docs = make_docs(train_data[:num_texts])
doc_bin = DocBin(docs=train_docs)
doc_bin.to_disk("./data/train.spacy")

valid_docs = make_docs(valid_data[:num_texts])
doc_bin = DocBin(docs=valid_docs)
doc_bin.to_disk("./data/valid.spacy")